In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [44]:
def update_nested_dict(base_dict, key_path, value):
    """Recursively update a nested dictionary given a dot-separated key."""
    keys = key_path.split(".")
    d = base_dict
    for k in keys[:-1]:  # Traverse the dictionary
        if k not in d:
            d[k] = {}  # Create nested dictionaries if they don't exist
        d = d[k]
    d[keys[-1]] = value  # Set the final value

DATASET = 'etth1'
MODEL = 'patchtst'
args={'config': f'config/ltsf/{DATASET}/{MODEL}.yaml', 
        'seed_everything':0,
       'data.data_manager.init_args.path': './datasets',
       'trainer.default_root_dir': './log_dir',
       'data.data_manager.init_args.dataset':f'{DATASET}',
       'data.data_manager.init_args.split_val': True,
       'trainer.max_epochs':50,
       'data.data_manager.init_args.context_length':96,
       'data.data_manager.init_args.prediction_length':96}

yaml_config_path = f'/home/kreffert/Probabilistic_LTSF/ProbTS/{args["config"]}'
with open(yaml_config_path, "r") as f:
    yaml_config = yaml.safe_load(f)
# Merge the CLI arguments into the YAML config
for key, value in args.items():
    if key == "config":  # Skip the config file path
        continue
    update_nested_dict(yaml_config, key, value)

In [45]:
from probts.data import DataManager, ProbTSDataModule

data_manager = DataManager(**yaml_config["data"]["data_manager"]["init_args"])
# data_manager.prepare_dataset()
data_module = ProbTSDataModule(data_manager= data_manager, 
                               batch_size= yaml_config["data"]['batch_size'],
                                test_batch_size= yaml_config["data"]['test_batch_size'],
                                num_workers= yaml_config["data"]['num_workers'])

train_loader = data_module.train_dataloader
val_loader = data_module.val_dataloader
test_loader = data_module.test_dataloader

Loading Long-term Dataset: etth1
val  pred_len: 96 : num_test_windows: 29
test  pred_len: 96 : num_test_windows: 29
Test context length: [96], prediction length: [96]
Validation context length: [96], prediction length: [96]
Training context length: [96], prediction lengths: [96]
Test rolling length: 96
Variable-specific normalization: True


/home/kreffert/Probabilistic_LTSF/ProbTS/probts/data/data_utils/get_datasets.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stamp['date'] = pd.to_datetime(df_stamp.date, format=data_format)


In [58]:
data_to_model_link_args = ["scaler",
                            "train_pred_len_list", ]
data_to_forecaster_link_args = [
    "target_dim",
    "history_length",
    "context_length",
    "prediction_length",
    "train_pred_len_list", 
    "lags_list",
    "freq",
    "time_feat_dim",
    "global_mean",
    "dataset"
]

for arg in data_to_forecaster_link_args:
    if arg in dir(data_manager):
        yaml_config['model']['forecaster']['init_args'][arg]=getattr(data_manager, arg)

for arg in data_to_model_link_args:
    if arg in dir(data_manager):
        yaml_config['model'][arg] = getattr(data_manager, arg)

In [65]:
from probts.model.forecast_module import ProbTSForecastModule
from probts.model.forecaster.point_forecaster import PatchTST
forecaster = PatchTST(**yaml_config["model"]['forecaster']['init_args'])
path="/home/kreffert/Probabilistic_LTSF/ProbTS/log_dir/etth1_PatchTST_CTX96_PRED96_seed0/ckpt/epoch=30-val_CRPS=0.335570.ckpt"
model = ProbTSForecastModule(forecaster=forecaster, 
                             **{k: v for k, v in yaml_config["model"].items() if k != 'forecaster'})
# model = model.load_from_checkpoint(path, **{k: v for k, v in yaml_config["model"].items() if k != 'forecaster'})

sampling_weight_scheme: none


In [66]:
print(type(model))

<class 'probts.model.forecast_module.ProbTSForecastModule'>


In [64]:
import pytorch_lightning as pl

# Initialize the PyTorch Lightning trainer
trainer = pl.Trainer(
    max_epochs=10,            # Specify the number of epochs
    #gpus=1,                   # Use a GPU, set to `None` or `0` for CPU
    precision=16,             # Mixed precision training, optional
    #accumulate_grad_batches=1 # Gradient accumulation, optional
)

# Train the model using the fit method
trainer.fit(
    model,   # Your validation DataLoader
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `ProbTSForecastModule`